In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from mezo.currency_utils import format_musd_currency_columns, get_token_price, format_currency_columns
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import SubgraphClient, Web3Client
from mezo.queries import MUSDQueries
load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [2]:
# import raw data
# raw_loans = SubgraphClient.get_subgraph_data(SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH, )
# raw_liquidations = get_liquidation_data()
# raw_troves_liquidated = get_trove_liquidated_data()

raw_loans = SubgraphClient.get_subgraph_data(
    SubgraphClient.BORROWER_OPS_SUBGRAPH,
    MUSDQueries.GET_LOANS,
    'troveUpdateds'
)

raw_liquidations = SubgraphClient.get_subgraph_data(
    SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH,
    MUSDQueries.GET_MUSD_LIQUIDATIONS,
    'liquidations'
)

raw_troves_liquidated = SubgraphClient.get_subgraph_data(
    SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH,
    MUSDQueries.GET_LIQUIDATED_TROVES,
    'troveLiquidateds'
)

🔍 Trying troveUpdateds query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 607 troveUpdateds records
🔍 Trying liquidations query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 2 liquidations records
🔍 Trying troveLiquidateds query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 2 troveLiquidateds records


In [3]:
# helpers
def clean_loan_data(raw, sort_col, date_cols, currency_cols):
    df = raw.copy().sort_values(by=sort_col, ascending=False)
    df = format_datetimes(df, date_cols)
    df = format_musd_currency_columns(df, currency_cols)
    df['count'] = 1
    df['id'] = range(1, len(df) + 1)

    return df

def find_coll_ratio(df, token_id):
    """Computes the collateralization ratio"""
    usd = get_token_price(token_id)
    df['coll_usd'] = df['coll'] * usd
    df['coll_ratio'] = (df['coll_usd']/df['principal'] ).fillna(0)

    return df

def get_loans_subset(df, operation: int, equals):
    """Create a df with only new, adjusted, or closed loans
    0 = opened, 1 = closed, 2 = adjusted
    note: operation = 2 also includes liquidated loans, so we have to remove those manually
    """
    df['operation'] = df['operation'].astype(int)
    if equals is True:
        adjusted = df.loc[df['operation'] == operation]
    elif equals is False:
        adjusted = df.loc[df['operation'] != operation]

    return adjusted

def process_liquidation_data(liquidations, troves_liquidated):
    # Merge raw liquidation data from two queries
    liquidation_df_merged = pd.merge(
        liquidations, 
        troves_liquidated, 
        how='left', 
        on='transactionHash_'
    )

    liquidation_df_merged = liquidation_df_merged[
        ['timestamp__x', 
        'liquidatedPrincipal', 
        'liquidatedInterest', 
        'liquidatedColl', 
        'borrower',
        'transactionHash_',
        'count_x'
        ]
    ]

    liquidations_df_final = liquidation_df_merged.rename(
        columns = {
            'timestamp__x': 'timestamp_', 
            'liquidatedPrincipal': 'principal', 
            'liquidatedInterest': 'interest',
            'liquidatedColl': 'coll',
            'count_x': 'count'
        }
    )

    liquidations_final = liquidations_df_final.copy()
    liquidations_final['coll'] = liquidations_final['coll'].astype(float)

    return liquidations_final

In [4]:
# clean raw data
loans = clean_loan_data(
    raw_loans, 
    sort_col='timestamp_', 
    date_cols=['timestamp_'], 
    currency_cols=['principal', 'coll', 'stake', 'interest']
)

loans = find_coll_ratio(loans, 'bitcoin')

liquidations = clean_loan_data(
    raw_liquidations,
    sort_col='timestamp_',
    date_cols=['timestamp_'],
    currency_cols=['liquidatedPrincipal', 'liquidatedInterest', 'liquidatedColl']
)

troves_liquidated = clean_loan_data(
    raw_troves_liquidated,
    sort_col='timestamp_',
    date_cols=['timestamp_'],
    currency_cols=['debt', 'coll']
)

# Create df for liquidated loans
liquidations_final = process_liquidation_data(liquidations, troves_liquidated)

# Create df's for new loans, closed loans, and adjusted loans and upload to BigQuery
new_loans = get_loans_subset(loans, 0, True)
closed_loans = get_loans_subset(loans, 1, True)
adjusted_loans = get_loans_subset(loans, 2, True) # Only adjusted loans (incl multiple adjustments from a single user)

## Remove liquidations from adjusted loans
liquidated_borrowers = liquidations_final['borrower'].unique()
adjusted_loans = adjusted_loans[~adjusted_loans['borrower'].isin(liquidated_borrowers)]

##################################

# Get latest loans
latest_loans = loans.drop_duplicates(subset='borrower', keep='first')

# Create df with only open loans
latest_open_loans = get_loans_subset(latest_loans, 1, False)

# Remove liquidated loans from list of latest loans w/o closed loans
latest_open_loans = latest_open_loans[~latest_open_loans['borrower'].isin(liquidated_borrowers)]

##################################

# Break down adjusted loan types for analysis
adjusted_loans = adjusted_loans.sort_values(by=['borrower', 'timestamp_'])
first_tx = adjusted_loans.groupby('borrower').first().reset_index()

adjusted_loans_merged = adjusted_loans.merge(
    first_tx[['borrower', 'principal', 'coll']], 
    on='borrower', 
    suffixes=('', '_initial')
)

## Loan increases
increased_loans = adjusted_loans_merged[adjusted_loans_merged['principal'] 
                                        > adjusted_loans_merged['principal_initial']].copy()
increased_loans['type'] = 1

## Collateral changes
coll_increased = adjusted_loans_merged[adjusted_loans_merged['coll'] 
                                       > adjusted_loans_merged['coll_initial']].copy()
coll_increased['type'] = 2

coll_decreased = adjusted_loans_merged[adjusted_loans_merged['coll'] 
                                       < adjusted_loans_merged['coll_initial']].copy()
coll_decreased['type'] = 3

## MUSD Repayments
principal_decreased = adjusted_loans_merged[adjusted_loans_merged['principal'] 
                                            < adjusted_loans_merged['principal_initial']].copy()
principal_decreased['type'] = 4

## Create final_adjusted_loans dataframe with type column
final_adjusted_loans = pd.concat([
    increased_loans,
    coll_increased, 
    coll_decreased,
    principal_decreased
], ignore_index=True)

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_51289/894492014.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['operation'] = df['operation'].astype(int)


In [6]:
new_loans

,timestamp_,borrower,principal,coll,stake,interest,operation,transactionHash_,block_number,count,id,coll_usd,coll_ratio
0,2025-08-10,0xaf4710372becbfbb2ce2719029b95d95d6b14ecd,2001.8,0.089798,0.089798,0.0,0,0x9d58ad353dae8be4469e09fd46b0d49323aecff04257...,2335614,1,1,10662.584108,5.326498
12,2025-08-09,0x77d780d49a609291666432953e94376cd5a2b7ce,2001.8,0.025920,0.025920,0.0,0,0x403ed2fc2d40502faf2e0347c3a648cf3fc78ecec587...,2305609,1,13,3077.799570,1.537516
18,2025-08-09,0x5eaa8bdc315608e8cf873cfaba05e42826f68c4d,100200.0,0.998776,0.998776,0.0,0,0x5d56ab218512f6041b8c66583d85851b5dd0c8051dcc...,2302940,1,19,118594.666162,1.183580
21,2025-08-08,0x76d00794c110061e56dbe28fabcefb72c864c24d,45245.0,0.967572,0.967572,0.0,0,0x978082398c6fefd3a89e9faf2b692cb082458d5c302b...,2297858,1,22,114889.505257,2.539275
36,2025-08-08,0x94d80146ae0ac239367358c5455f72caf134abec,2202.0,0.025000,0.025000,0.0,0,0x3f6956c192b1e971b6d548f07d411ad92b56b7fabd48...,2287732,1,37,2968.490428,1.348088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,2025-05-23,0x9a8e1ffc29329e2f28f65278b8a569ef5f06c1aa,2202.0,0.023000,0.023000,0.0,0,0x6148626ebed002f6788684bf6c50908178b39fce7c40...,445371,1,601,2731.020000,1.240245
602,2025-05-23,0xa21b23e5f07e5e28cb42d09502227eb75b0b64b5,2202.0,0.024500,0.024500,0.0,0,0xad5d73fbe166ae0f93aa56912483381d5f2c08cf1b6c...,445211,1,603,2909.130000,1.321131
604,2025-05-23,0x984600f00edbc969d2a746a554c8e91d358ef843,2202.0,0.024218,0.024218,0.0,0,0xeceb4fb1bd666fd3a67e630ad0a523cabc7825b38a00...,444497,1,605,2875.644188,1.305924
605,2025-05-22,0xe4c207f76c5c4bd6194bc299c82ed0a8b3021539,2202.0,0.040000,0.040000,0.0,0,0xffda614cd610859c4f367e5fb3944368348a13f098e5...,420170,1,606,4749.600000,2.156948


In [5]:
GET_REDEMPTIONS = """
query getRedemptions($skip: Int!) {
    redemptions(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
  ) {
    timestamp_
    actualAmount
    attemptedAmount
    collateralFee
    collateralSent
    transactionHash_
    block_number
  }
}
"""

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

In [7]:
def get_redemptions():
    """
    Get redemption data for troves from the MUSD Trove Manager subgraph
    """
    musd = SubgraphClient(
        url=SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH, 
        headers=SubgraphClient.SUBGRAPH_HEADERS
    )
    
    print("🔍 Trying redemptions query...")
    try:
        redemptions_data = musd.fetch_subgraph_data(
            GET_REDEMPTIONS, 
            'redemptions'
        )
    
        if redemptions_data:
            redemptions_df = pd.DataFrame(redemptions_data)
            print(f"✅ Found {len(redemptions_df)} redemption records")
            
            return redemptions_df
        else:
            print("⚠️ redemptions query returned no data")
    except Exception as e:
        print(f"❌ redemptions query failed: {e}")

In [8]:
raw_redemptions = get_redemptions()

🔍 Trying redemptions query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 2 redemption records


In [10]:
raw_redemptions

,timestamp_,actualAmount,attemptedAmount,collateralFee,collateralSent,transactionHash_,block_number
0,1754690451,1803385251291886491445,1803385251291886491445,115674770606549,15423302747539974,0x4bdfb12a4c7d714fe62e2b2367767c6c9718a202bab8...,2296861
1,1748363330,1000000000000000000,1000000000000000000,68040765191,9072102025476,0x000756bb3b11475c2fef560c307a13853aace3ec1ec6...,541150


In [9]:
redemptions_clean = clean_loan_data(
    raw_redemptions,
    sort_col='timestamp_',
    date_cols=['timestamp_'],
    currency_cols=['actualAmount', 'attemptedAmount', 'collateralFee', 'collateralSent']
)
redemptions_clean.head()

,timestamp_,actualAmount,attemptedAmount,collateralFee,collateralSent,transactionHash_,block_number,count,id
0,2025-08-08,1803.385251,1803.385251,1.156748e-04,0.015423,0x4bdfb12a4c7d714fe62e2b2367767c6c9718a202bab8...,2296861,1,1
1,2025-05-27,1.000000,1.000000,6.804077e-08,0.000009,0x000756bb3b11475c2fef560c307a13853aace3ec1ec6...,541150,1,2


In [11]:
refinanced_loans = get_loans_subset(loans, 3, True)
refinanced_loans

,timestamp_,borrower,principal,coll,stake,interest,operation,transactionHash_,block_number,count,id,coll_usd,coll_ratio
5,2025-08-10,0x043b223fd4804ca5f9151c1d44250bb7799287ac,7622.728312,0.360833,0.360833,1.163683,3,0x76dcef47ea1e3b638a7c9b4c9b2fbf0e77cfa563b16f...,2333615,1,6,42845.363486,5.620739
24,2025-08-08,0xce96ac0be6a54d5c5bba49e90cc7f965581e5fde,61208.456171,0.802517,0.802517,5.629293,3,0x25266ad8eb4355b72d97cfa9180a2d24f2e7f928c3c3...,2297108,1,25,95290.811923,1.556824
33,2025-08-08,0xff0b6ffc4507812f5380062d91a79e464508bde1,16219.204769,0.774984,0.774984,7.844990,3,0x589ae293355272425c0cbece705e40d985591e9d4cdf...,2289078,1,34,92021.600160,5.673620
62,2025-08-06,0x46633b491c0dd7b245f47da22855f33fa20a4e06,14218.806795,0.400000,0.400000,3.985552,3,0x123cb2ca0ddff9d634b099ecc18f352b30b5f40b0e95...,2243673,1,63,47496.000000,3.340365
65,2025-08-06,0x46633b491c0dd7b245f47da22855f33fa20a4e06,10212.002797,0.200000,0.200000,3.985187,3,0x24178366b3c0a56553eeaf91687c3208cc8c2af11282...,2243649,1,66,23748.000000,2.325499
69,2025-08-06,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,23230.207728,0.303000,0.303000,0.033843,3,0xa371e2bbe1ffbc274a2393c9bbac5d1c769c2cae5c35...,2241834,1,70,35978.220000,1.548769
72,2025-08-06,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,13215.602601,0.240000,0.240000,0.003408,3,0xcf70cdb3615e88a6420bd60c8ff4ae3bff0a3402b894...,2240680,1,73,28497.600000,2.156360
76,2025-08-06,0xce96ac0be6a54d5c5bba49e90cc7f965581e5fde,32238.406874,0.590688,0.590688,2.367555,3,0xdd677eb9e7051d6a29d69048082ae42e74566b605fe5...,2240509,1,77,70138.310846,2.175613
84,2025-08-06,0xb41854682fe2f21c1225682726eaada484b8a5f2,6811.306233,0.156483,0.156483,0.826244,3,0x1186967c27f1f1293300b5b896f1f652624087b26da7...,2237116,1,85,18580.843933,2.727941
89,2025-08-05,0x03cfb083b4547e6b4866ff51a05c5cab9a22da93,2803.120780,0.047499,0.047499,1.297715,3,0x2240897b7509da642b3e648e1afcb1d2693c02ac928b...,2221039,1,90,5640.081873,2.012072


# Fees

In [12]:
# Get borrowing rate

w3 = Web3Client('BorrowerOperations')
borrower_ops = w3.load_contract()
borrowing_rate = borrower_ops.functions.borrowingRate().call()

In [13]:
# Get issuance fee

borrowing_rate = (borrowing_rate/1e18)
new_loans['issuance_fee'] = new_loans['principal'] * borrowing_rate

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_51289/40239977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_loans['issuance_fee'] = new_loans['principal'] * borrowing_rate


In [14]:
# Get gas comp

trove = Web3Client('troveManager')
troves = trove.load_contract()
gas_comp = troves.functions.MUSD_GAS_COMPENSATION().call()
gas_comp = gas_comp / 1e18

In [15]:
# Get refinancing rate

refinance_fee_pctage = borrower_ops.functions.refinancingFeePercentage().call()
borrowing_fee_pctage = borrowing_rate * 100
refinance_fee = refinance_fee_pctage/100
refinance_rate = borrowing_fee_pctage * refinance_fee
final_refinance_rate = refinance_rate/100

In [16]:
# Calculate refinance fees and add to df

refinanced_loans['fees'] = (refinanced_loans['principal'] + refinanced_loans['interest'] - gas_comp) * final_refinance_rate

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_51289/3258910165.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refinanced_loans['fees'] = (refinanced_loans['principal'] + refinanced_loans['interest'] - gas_comp) * final_refinance_rate


In [43]:
redemptions_clean

,timestamp_,actualAmount,attemptedAmount,collateralFee,collateralSent,transactionHash_,block_number,count,id
0,2025-08-08,1803.385251,1803.385251,1.156748e-04,0.015423,0x4bdfb12a4c7d714fe62e2b2367767c6c9718a202bab8...,2296861,1,1
1,2025-05-27,1.000000,1.000000,6.804077e-08,0.000009,0x000756bb3b11475c2fef560c307a13853aace3ec1ec6...,541150,1,2


In [17]:
GET_BORROW_FEES = """
query getBorrowFees ($skip: Int!) {
  borrowingFeePaids (
    orderBy: timestamp_
    orderDirection: desc
    first: 1000
    skip: $skip
  ){
    timestamp_
    fee
    borrower
    transactionHash_
  }
}"""

In [18]:
# Fetch borrow fees from subgraph 

musd = SubgraphClient(
    url=SubgraphClient.BORROWER_OPS_SUBGRAPH, 
    headers= SubgraphClient.SUBGRAPH_HEADERS
)
fees =  musd.fetch_subgraph_data(GET_BORROW_FEES, 'borrowingFeePaids')

if fees:
    fees = pd.DataFrame(fees)
    print(f"✅ Found {len(fees)} fee records")
else:
    print("⚠️ Query returned no data")

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 510 fee records


In [41]:
test = pd.merge(raw_redemptions, fees, how='left')
test['transactionHash_'][1]

'0x000756bb3b11475c2fef560c307a13853aace3ec1ec65b296abf0320c864a98f'

In [26]:
# MOVE THIS CODE CHUNK TO HEX
#  Add fees to loans df
########################

loan_fees = fees.copy()
loans_with_fees = pd.merge(loans, loan_fees, how='left', on='transactionHash_')

timestamp__x        607
borrower_x          607
principal           607
coll                607
stake               607
interest            607
operation           607
transactionHash_    607
block_number        607
count               607
id                  607
coll_usd            607
coll_ratio          607
timestamp__y        510
fee                 510
borrower_y          510
dtype: int64

In [ ]:
# Create daily dataframe
daily_new_loans = new_loans.groupby(['timestamp_']).agg(
    loans_opened = ('count', 'sum'),
    borrowers = ('borrower', lambda x: x.nunique()),
    principal = ('principal', 'sum'),
    collateral = ('coll', 'sum'),
    interest = ('interest', 'sum')
).reset_index()

daily_closed_loans = closed_loans.groupby(['timestamp_']).agg(
    loans_closed = ('count', 'sum'),
    borrowers_who_closed = ('borrower', lambda x: x.nunique())
).reset_index()

daily_new_and_closed_loans = pd.merge(daily_new_loans, daily_closed_loans, how = 'outer', on = 'timestamp_').fillna(0)
daily_new_and_closed_loans[['loans_opened', 'borrowers', 'loans_closed', 'borrowers_who_closed']] = daily_new_and_closed_loans[['loans_opened', 'borrowers', 'loans_closed', 'borrowers_who_closed']].astype('int')      
daily_adjusted_loans = adjusted_loans.groupby(['timestamp_']).agg(
    loans_adjusted = ('count', 'sum'),
    borrowers_who_adjusted = ('borrower', lambda x: x.nunique())
).reset_index()

daily_loan_data = pd.merge(daily_new_and_closed_loans, daily_adjusted_loans, how='outer', on='timestamp_').fillna(0)
daily_loan_data[['loans_adjusted', 'borrowers_who_adjusted']] = daily_loan_data[['loans_adjusted', 'borrowers_who_adjusted']].astype(int)

daily_balances = latest_loans.groupby(['timestamp_']).agg(
    musd = ('principal', 'sum'),
    interest = ('interest', 'sum'),
    collateral = ('coll', 'sum')
).reset_index()

daily_balances = daily_balances.rename(
    columns={'musd': 'net_musd', 
             'interest': 'net_interest',
             'collateral': 'net_coll'}
)

daily_loans_merged = pd.merge(daily_loan_data, daily_balances, how='outer', on='timestamp_')

cols = {
    'timestamp_': 'date', 
    'principal': 'gross_musd', 
    'collateral': 'gross_coll', 
    'interest': 'gross_interest',
    'borrowers_who_closed': 'closers', 
    'borrowers_who_adjusted': 'adjusters'
}

daily_loans_merged = daily_loans_merged.rename(columns = cols)

daily_musd_final = add_rolling_values(daily_loans_merged, 30, ['net_musd', 'net_interest', 'net_coll']).fillna(0)
daily_musd_final_2 = add_cumulative_columns(daily_musd_final, ['net_musd', 'net_interest', 'net_coll'])
daily_musd_final_3 = add_pct_change_columns(daily_musd_final_2, ['net_musd', 'net_interest', 'net_coll'], 'daily').fillna(0)
final_daily_musd = daily_musd_final_3.replace([float('inf'), -float('inf')], 0)
final_daily_musd['date'] = pd.to_datetime(final_daily_musd['date']).dt.strftime('%Y-%m-%d')

In [21]:
token = Web3Client('MUSD')
t = token.load_contract()
t.all_functions()
t.functions.mint(0).call()

MismatchedABI: 
ABI Not Found!
No element named `mint` with 1 argument(s).
Provided argument types: (int)
Provided keyword argument types: {}

Tried to find a matching ABI element named `mint`, but encountered the following problems:
Signature: mint(address,uint256), type: function
Expected 2 argument(s) but received 1 argument(s).


In [ ]:
import requests
import pandas as pd
import time
from typing import Optional, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_paginated_api_data(
    endpoint: str, 
    items_key: str = 'items',  # Key to extract items from response
    page_size: int = 1000,  # Items per page
    page_size_param: str = 'limit',  # Parameter name for page size
    max_retries: int = 3,
    timeout: int = 30,
    rate_limit_delay: float = 0.5,
    additional_params: Optional[Dict[str, Any]] = None
) -> pd.DataFrame:
    """
    Fetch data from the specified API endpoint with pagination.
    
    Args:
        endpoint: API endpoint path
        items_key: Key in response JSON that contains the items
        page_size: Number of items to request per page (default 1000)
        page_size_param: Parameter name for page size ('limit', 'per_page', etc.)
        max_retries: Maximum number of retries for failed requests
        timeout: Request timeout in seconds
        rate_limit_delay: Delay between requests to avoid rate limiting
        additional_params: Additional query parameters to include
    
    Returns:
        DataFrame with all fetched data
    """
    base_url = 'https://api.explorer.mezo.org/api/v2'
    url = f"{base_url}/{endpoint}"
    logger.info(f"Fetching data from: {url} with page size: {page_size}")
    
    all_data = []
    next_page_params = {}
    page_count = 0
    total_items = 0
    
    # Set up initial parameters
    if additional_params:
        next_page_params.update(additional_params)
    
    # Add page size parameter
    next_page_params[page_size_param] = page_size
    
    while True:
        retry_count = 0
        success = False
        
        # Retry logic
        while retry_count < max_retries and not success:
            try:
                logger.info(f"Fetching page {page_count + 1}, items so far: {total_items}")
                
                # Make request with current params
                response = requests.get(
                    url, 
                    params=next_page_params, 
                    timeout=timeout
                )
                
                # Check status code
                if response.status_code == 429:  # Rate limited
                    logger.warning("Rate limited, waiting longer...")
                    time.sleep(rate_limit_delay * 5)
                    retry_count += 1
                    continue
                elif response.status_code != 200:
                    logger.error(f"HTTP {response.status_code}: {response.text}")
                    raise Exception(f"Failed to fetch data: {response.status_code}")
                
                # Parse response
                data = response.json()
                items = data.get(items_key, [])
                
                if not items:
                    logger.info("No more items found, stopping pagination")
                    break
                
                # Convert to DataFrame and append
                df_chunk = pd.json_normalize(items)
                all_data.append(df_chunk)
                
                total_items += len(items)
                page_count += 1
                
                # Check for next page
                new_page_params = data.get("next_page_params")
                if not new_page_params:
                    logger.info("No next_page_params found, stopping pagination")
                    break
                
                # Update pagination params but keep page size
                next_page_params.update(new_page_params)
                next_page_params[page_size_param] = page_size  # Ensure page size is maintained
                
                success = True
                
                # Rate limiting delay
                time.sleep(rate_limit_delay)
                
            except requests.exceptions.Timeout:
                logger.warning(f"Timeout on retry {retry_count + 1}")
                retry_count += 1
                time.sleep(2 ** retry_count)  # Exponential backoff
                
            except requests.exceptions.RequestException as e:
                logger.error(f"Request failed on retry {retry_count + 1}: {e}")
                retry_count += 1
                time.sleep(2 ** retry_count)
                
            except Exception as e:
                logger.error(f"Unexpected error: {e}")
                raise
        
        if not success:
            raise Exception(f"Failed to fetch data after {max_retries} retries")
        
        # Break if we successfully processed a page with no next_page_params
        if not new_page_params:
            break
    
    logger.info(f"Completed! Fetched {total_items} items across {page_count} pages")
    
    # Combine all DataFrames
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        logger.info(f"Final DataFrame shape: {final_df.shape}")
        return final_df
    else:
        logger.warning("No data was fetched")
        return pd.DataFrame()


def fetch_single_page_data(endpoint: str, timeout: int = 30) -> pd.DataFrame:
    """
    Fetch data from a single page endpoint (non-paginated).
    """
    base_url = 'https://api.explorer.mezo.org/api/v2'
    url = f"{base_url}/{endpoint}"
    logger.info(f"Fetching single page from: {url}")
    
    try:
        response = requests.get(url, timeout=timeout)
        
        if response.status_code != 200:
            raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")
        
        data = response.json()
        return pd.json_normalize(data)
        
    except requests.exceptions.Timeout:
        raise Exception(f"Request timed out after {timeout} seconds")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")

INFO:__main__:Fetching data from: https://api.explorer.mezo.org/api/v2/your-endpoint-here with page size: 1000
INFO:__main__:Fetching page 1, items so far: 0
ERROR:__main__:HTTP 400: {"message":"Params 'module' and 'action' are required parameters","result":null,"status":"0"}
ERROR:__main__:Unexpected error: Failed to fetch data: 400


Error fetching data: Failed to fetch data: 400


In [44]:
musd_token_address = '0xdD468A1DDc392dcdbEf6db6e34E89AA338F9F186'

try:
    # For paginated endpoints with custom page size
    df = fetch_paginated_api_data(
        endpoint=f"tokens/{musd_token_address}/transfers?type=token_minting",
        items_key="items",  # Adjust based on your API response structure
        page_size=1000,  # Request 1000 items per page instead of default 50
        page_size_param="page_size",  # Try "limit", "per_page", "page_size", etc.
        timeout=60,
        rate_limit_delay=0.5
    )
    print(f"Successfully fetched {len(df)} records")
    
except Exception as e:
    print(f"Error fetching data: {e}")

INFO:__main__:Fetching data from: https://api.explorer.mezo.org/api/v2/tokens/0xdD468A1DDc392dcdbEf6db6e34E89AA338F9F186/transfers?type=token_minting with page size: 1000
INFO:__main__:Fetching page 1, items so far: 0
INFO:__main__:Fetching page 2, items so far: 50
INFO:__main__:Fetching page 3, items so far: 100
INFO:__main__:Fetching page 4, items so far: 150
INFO:__main__:Fetching page 5, items so far: 200
INFO:__main__:Fetching page 6, items so far: 250
INFO:__main__:Fetching page 7, items so far: 300


KeyboardInterrupt: 